# Creación de Imágenes Temporales Asociadas a una Frecuencia de Muestreo Específica
En este notebook se realizará el mismo procedimiento del archivo Creación_imágenes_fisura, sólo que ahora en el conjunto de datos asociados a pernoflojo.

## Importaciones
Se realizan las importaciones necesarias para llevar a cabo la obtención y procesamiento de datos.

In [42]:
from google.colab import drive
from os import listdir
from os.path import isfile, join
from scipy.io import loadmat
from random import randint

import pandas as pd
import numpy as np
import random

import os

## Obtención de Datos
Nos conectamos con Google Drive para obtener el dataset general y segmentar los datos relacionados a pernoflojo.

In [43]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [44]:
dataset_datos_dir = 'drive/MyDrive/DATOS_EXPERIMENTALES_JACKET/DATOS'

In [45]:
dataset_datos_files = [dataset_datos_dir+'/'+filename for filename in listdir(dataset_datos_dir) if isfile(join(dataset_datos_dir, filename))]

In [46]:
# Creamos listas en donde se almacenarán los archivos relacionados a experimentos de fisura y experimentos de perno flojo respectivamente
datos_fisura = []
datos_pernoflojo = []

# Iteramos los nombres de todos los archivos en la lista general previamente creada
for filename in listdir(dataset_datos_dir):
  # Obtenemos el nombre del archivo tal cual, sin direcciones previas
  if isfile(join(dataset_datos_dir, filename)):
    nombre_archivo = filename.split('.')[0]
    # El estado de fisura se asocia al número 3 en los nombres de los archivos, si cumple la condición el archivo, guardamos en la lista datos_fisura
    if (nombre_archivo.split('_')[0] == '3'):
      datos_fisura.append(dataset_datos_dir + '/' + filename)
    # El estado de fisura se asocia al número 4 en los nombres de los archivos, si cumple la condición el archivo, guardamos en la lista datos_pernoflojo
    elif (nombre_archivo.split('_')[0] == '4'):
      datos_pernoflojo.append(dataset_datos_dir + '/' + filename)

### Dataset Pernoflojo
Llevamos el dataset pernoflojo a un DataFrame en donde estarán todos los experimentos asociados a este tipo de daño apilados.

In [47]:
# Se crean 2 listas, una asociada al número de experimento y otra asociada al nivel de white noise presente durante el experimento 
numero_experimentos_pernoflojo = []
amplitud_experimentos_pernoflojo = []

# Iteramos los nombres de los archivos dentro del conjunto que contiene los datos de fisura
for nombre_archivo in datos_pernoflojo:
  # Obtenemos el nombre del experimento
  nombre_experimento = nombre_archivo.split('/')[4]
  # Obtenemos el número del experimento
  numero_experimento = nombre_experimento.split('_')[1]
  # Obtenemos el nivel de white noise del experimento
  amplitud_experimento = nombre_experimento.split('.')[0].split('_')[2].replace('A', '')
  # Agregamos el número del experimento a la lista numero_experimentos_pernoflojo creada anteriormente
  numero_experimentos_pernoflojo.append(int(numero_experimento))
  # Agregamos el nivel de white noise del experimento a la lista amplitud_experimentos_pernoflojo creada anteriormente
  if amplitud_experimento == '05':
    amplitud_experimento = 0.5
    amplitud_experimentos_pernoflojo.append(amplitud_experimento)
  else:
    amplitud_experimentos_pernoflojo.append(int(amplitud_experimento))

In [48]:
# Creamos un DataFrame vacío
df_pernoflojo = pd.DataFrame()

# Iteramos por rango la lista datos_fisura
for indice in range(len(datos_pernoflojo)):
  # Obtenemos la dirección dentro de la iteración correspondiente
  direccion = datos_pernoflojo[indice]
  # Transformamos el .mat a un formato en el que se pueda transformar a DataFrame
  mat = loadmat(direccion)
  df = pd.DataFrame(mat['data'])
  # Para todos los valores del experimento, creamos columnas en donde sus valores se asocian con el nivel de wn y el # de experimento
  df['#_exp'] = numero_experimentos_pernoflojo[indice]
  df['amplitud'] = amplitud_experimentos_pernoflojo[indice]
  # Colocamos al final del DataFrame df_fisura creado el nuevo DataFrame 
  df_pernoflojo = pd.concat([df_pernoflojo, df], axis = 0)

In [49]:
# Revisamos el conjunto de datos obtenido para pernoflojo
df_pernoflojo.head(8)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,#_exp,amplitud
0,0.000168,0.000204,0.000213,0.000135,0.000201,0.000099,0.000108,0.000189,0.000212,0.000093,0.000257,0.000196,0.000173,0.000029,0.000316,0.000202,0.000010,0.000085,0.000151,0.000136,0.000116,0.000099,0.000054,0.000111,1,0.5
1,0.000122,0.000277,0.000213,0.000180,0.000230,0.000172,0.000068,0.000213,0.000214,0.000095,0.000272,0.000172,0.000173,0.000050,0.000292,0.000178,0.000021,0.000022,0.000203,0.000139,0.000076,0.000164,0.000089,0.000076,1,0.5
2,0.000099,0.000214,0.000128,0.000153,0.000217,0.000100,0.000064,0.000174,0.000225,0.000116,0.000245,0.000202,0.000140,0.000012,0.000309,0.000256,0.000023,0.000089,0.000225,0.000162,0.000098,0.000154,0.000091,0.000092,1,0.5
3,0.000150,0.000258,0.000209,0.000163,0.000213,0.000120,0.000066,0.000157,0.000220,0.000096,0.000229,0.000203,0.000157,0.000057,0.000292,0.000260,0.000062,0.000077,0.000212,0.000178,0.000123,0.000150,0.000056,0.000130,1,0.5
4,0.000194,0.000244,0.000194,0.000201,0.000202,0.000127,0.000133,0.000197,0.000212,0.000110,0.000266,0.000199,0.000181,0.000040,0.000336,0.000209,0.000092,0.000097,0.000183,0.000170,0.000114,0.000127,0.000064,0.000087,1,0.5
5,0.000156,0.000287,0.000188,0.000119,0.000234,0.000127,0.000102,0.000198,0.000216,0.000097,0.000280,0.000222,0.000140,0.000058,0.000296,0.000212,0.000061,0.000097,0.000231,0.000156,0.000090,0.000155,0.000076,0.000091,1,0.5
6,0.000150,0.000230,0.000198,0.000239,0.000247,0.000150,0.000052,0.000178,0.000255,0.000115,0.000235,0.000189,0.000216,0.000072,0.000299,0.000233,0.000060,0.000124,0.000225,0.000157,0.000133,0.000153,0.000060,0.000087,1,0.5
7,0.000130,0.000230,0.000207,0.000116,0.000212,0.000108,0.000096,0.000178,0.000200,0.000130,0.000260,0.000203,0.000216,0.000057,0.000329,0.000261,0.000031,0.000045,0.000236,0.000178,0.000081,0.000160,0.000079,0.000115,1,0.5


In [50]:
# Si dividimos para 99097 el df_pernoflojo deberíamos obtener 20 experimentos en total
len(df_pernoflojo)/99097

20.0

#### Procesamiento de Datos
Creamos las imágenes de (24, 16, 16) a partir de los datos del df_pernoflojo.

In [74]:
# Establecemos 2 listas, una con los posibles experimentos y otra con los posibles niveles de white noise, lo cual nos ayudará a segmentar los datos
experimentos = [1, 2, 3, 4, 5]
wns = [0.5, 1, 2, 3]
# Creamos una lista en donde pondremos todas las imágenes creadas, segmentadas por experimento y white noise
arreglo_imagenes_por_experimento_pernoflojo = []

# Iteramos por experimento y por nivel de white noise
for i in experimentos:
  for j in wns:
    # Creamos filtros que nos permiten identificar el experimento por su número asociado y por su nivel de white noise dentro del conjunto de datos
    filter1 = df_pernoflojo["#_exp"] == i
    filter2 = df_pernoflojo["amplitud"] == j
    # Filtramos y deshacemos las filas que no corresponden a la búsqueda
    dataset_experimento = df_pernoflojo.where(filter1 & filter2).dropna()
    # Creamos arreglo en donde almacenaremos listas de imágenes asociadas a cada columna en un experimento individual
    arreglo_matrices_asociadas = []

    # En el conjunto filtrado, iteramos por cada columna (de la columna 0 a la 23)
    for n in range(24):
      # Establecemos un límite inferior y un límite superior
      indice_inicio = 0
      indice_fin = 256
      # Del conjunto filtrado, obtenemos un subconjunto con muestras cada 6 pasos
      columna_cada_6 = dataset_experimento[n][::6]
      # Creamos una lista en donde guardaremos todas las imágenes creadas dentro de la columna en la que se esta iterando
      matrices_columna = []
    
      # Recorremos en valores de 256 la columna del subconjunto creado para generar las imágenes 16x16 correspondientes 
      while indice_fin < len(columna_cada_6):
        # Guardamos el conjunto de 256 datos dentro de un arreglo
        vector = columna_cada_6[indice_inicio:indice_fin]
        # Redimensionamos el arreglo (de 1x256 a 16x16)
        matriz = vector.to_numpy().reshape((16, 16))
        # Agregamos la imagen dentro de la lista matrices_columna
        matrices_columna.append(matriz)
        # Establecemos el nuevo límite inferior y superior en donde se encontrarán los datos de mi nueva imagen 16x16
        indice_inicio = indice_fin
        indice_fin = indice_fin + 256
      
      # Agregamos las listas de imágenes 16x16 asociadas a una columna a arreglo_matrices_asociadas
      arreglo_matrices_asociadas.append(matrices_columna)
    
    # Agregamos arreglo_matrices_asociadas a arreglos_imagenes_por_experimento, en donde tenemos listas de listas asociadas a un experimento individual
    arreglo_imagenes_por_experimento_pernoflojo.append(arreglo_matrices_asociadas)

In [52]:
len(arreglo_imagenes_por_experimento_pernoflojo)

20

In [53]:
len(arreglo_imagenes_por_experimento_pernoflojo[0])

24

In [54]:
len(arreglo_imagenes_por_experimento_pernoflojo[0][0])

64

In [55]:
arreglo_imagenes_por_experimento_pernoflojo[0][0][0].size

256

In [56]:
arreglo_imagenes_por_experimento_pernoflojo[0][0][0].shape

(16, 16)

In [65]:
# Creamos un arreglos general para todas las imágenes de dimensionamiento (24, 16, 16)
arreglo_imagenes_pernoflojo = []

# Iteramos cada experimento en la lista de experimentos creada previamente
for experimento_i in arreglo_imagenes_por_experimento_pernoflojo:
  # Iteramos en un rango de 64 para usarlo posteriormente
  for idx in range(64):
    # Para cada iteración se crea el arreglo en donde se agregaran las capas correctas de la imagen de dimensionamniento (24, 16, 16)
    imagen = []
    # Iteramos para cada fila en el experimento (son 24 filas)
    for fila in experimento_i:
      # Asociamos por índice los valores de cada lista de imágenes para establecer la relación correcta y agregamos al arreglo creado
      capa_imagen = fila[idx]
      imagen.append(capa_imagen)

    # Transformamos la lista de 24 capas a un arreglo numérico
    arreglo_imagenes_pernoflojo.append(np.array(imagen))

In [66]:
# Revisamos el número de imágenes creadas
len(arreglo_imagenes_pernoflojo) 

1280

In [67]:
# Revisamos el dimensionamiento y tipo de un elemento aleatorio en la lista de imágenes
indice_imagen_pernoflojo = randint(0, len(arreglo_imagenes_pernoflojo) - 1)
arreglo_imagenes_pernoflojo[indice_imagen_pernoflojo].shape, type(arreglo_imagenes_pernoflojo[0]), indice_imagen_pernoflojo

((24, 16, 16), numpy.ndarray, 427)

#### Verificación
Verificamos que se hayan obtenido las imágenes correctamente sin combinar datos de otros experimentos ajenos al experimento procesado.

In [76]:
# Establecemos los valores correspondientes a variables que nos van a ayudar con la localización de la imagen a comparar.
prueba_exp = random.choice(experimentos) # número de experimento
prueba_wn = random.choice(wns) # nivel de white noise
prueba_n = random.randint(1, 64) # índice de la imagen creada por experimento (en cada experimento individual se crean 64 imágenes)

In [77]:
# Creamos un diccionario que nos va a ayudar a relacionar la primera imagen con la segunda imagen
diccionario = {}
rango_inferior = 1
rango_superior = 64

# Creamos claves basados en el número de experimento y el nivel de white noise, en donde la clave será el intervalo en donde se encuentra la imagen 1
# a partir de los datos de la imagen 2
for i in experimentos:
  for j in wn:
    diccionario[(i, j)] = [rango_inferior, rango_superior]
    rango_inferior = rango_inferior + 64
    rango_superior = rango_superior + 64

In [63]:
def crear_subconjunto_prueba_pernoflojo(prueba_exp, prueba_wn):
  """
  Creamos el subconjunto de paso 6 a partir del dataset ingresado
  """
  prueba_filtro_1 = df_pernoflojo["#_exp"] == prueba_exp
  prueba_filtro_2 = df_pernoflojo["amplitud"] == prueba_wn
  prueba_subconjunto_experimento = df_pernoflojo.where(prueba_filtro_1 & prueba_filtro_2).dropna()

  prueba_subconjunto_paso6 = prueba_subconjunto_experimento[::6]

  return prueba_subconjunto_paso6

In [79]:
prueba_subconjunto_paso6_pernoflojo = crear_subconjunto_prueba_pernoflojo(prueba_exp, prueba_wn)

In [84]:
def crear_imagen_comparacion(prueba_n, prueba_subconjunto_paso_6):
  """
  Creamos la imagen_2 para la comparación
  """
  inicio = (prueba_n - 1)*256
  fin = prueba_n*256

  imagen = prueba_subconjunto_paso_6[inicio:fin]
  matriz = []

  for i in range(24):
    matriz.append(imagen[i].to_numpy().reshape((16, 16)))
  
  return np.array(matriz)

In [85]:
def obtener_imagen_original(prueba_exp, prueba_wn, diccionario, arreglo_imagenes,prueba_n):
  """
  Obtención de imagen_1 asociada a los parámetros escogidos para la imagen_2
  """
  rango = diccionario[(prueba_exp, prueba_wn)]

  return arreglo_imagenes[rango[0]-1:rango[1]-1][prueba_n-1]

In [86]:
# Almacenamos las imágenes en sus respectivas variables
imagen_1_pernoflojo = obtener_imagen_original(prueba_exp, prueba_wn, diccionario, arreglo_imagenes_pernoflojo, prueba_n)
imagen_2_pernoflojo = crear_imagen_comparacion(prueba_n, prueba_subconjunto_paso6_pernoflojo)

In [87]:
# Chequeamos los parámetros creados para ambas imágenes
print("Número de experimento: ", prueba_exp)
print("Nivel de White Noise:  ", prueba_wn)
print("Número de imagen:      ", prueba_n)

Número de experimento:  2
Nivel de White Noise:   3
Número de imagen:       1


In [93]:
# Verificamos si la imagen_1 es igual a la imagen_2
(imagen_1_pernoflojo == imagen_2_pernoflojo).all()

True

In [94]:
imagen_1_pernoflojo

array([[[ 4.77101280e-04,  1.54725600e-04,  5.21515200e-05, ...,
          2.96375520e-04,  1.84032480e-04, -7.97294400e-05],
        [ 5.07018720e-04,  8.93956800e-05, -7.66766400e-05, ...,
          8.63428800e-05, -2.25653280e-04, -4.30958400e-05],
        [ 2.58520800e-04,  5.46705120e-04,  3.76969440e-04, ...,
          4.34972640e-04,  5.21515200e-05, -2.12831520e-04],
        ...,
        [ 8.02372800e-05,  2.32266720e-04,  2.96375520e-04, ...,
         -3.33268800e-05,  6.97008000e-06,  1.46177760e-04],
        [-3.21057600e-05, -4.37064000e-05,  2.03570400e-04, ...,
          1.29692640e-04,  4.90533600e-04,  4.17876960e-04],
        [ 2.38982880e-04,  6.13099200e-05,  7.59633600e-05, ...,
          6.97008000e-06, -2.25042720e-04,  1.25418720e-04]],

       [[ 2.06667238e-04,  2.35308004e-04,  6.89478100e-05, ...,
          3.94965040e-04,  1.92042166e-04,  1.03682356e-04],
        [ 2.59683124e-04,  3.66933652e-04,  2.68214416e-04, ...,
          3.18183412e-04,  5.23543798e

In [95]:
imagen_2_pernoflojo

array([[[ 4.77101280e-04,  1.54725600e-04,  5.21515200e-05, ...,
          2.96375520e-04,  1.84032480e-04, -7.97294400e-05],
        [ 5.07018720e-04,  8.93956800e-05, -7.66766400e-05, ...,
          8.63428800e-05, -2.25653280e-04, -4.30958400e-05],
        [ 2.58520800e-04,  5.46705120e-04,  3.76969440e-04, ...,
          4.34972640e-04,  5.21515200e-05, -2.12831520e-04],
        ...,
        [ 8.02372800e-05,  2.32266720e-04,  2.96375520e-04, ...,
         -3.33268800e-05,  6.97008000e-06,  1.46177760e-04],
        [-3.21057600e-05, -4.37064000e-05,  2.03570400e-04, ...,
          1.29692640e-04,  4.90533600e-04,  4.17876960e-04],
        [ 2.38982880e-04,  6.13099200e-05,  7.59633600e-05, ...,
          6.97008000e-06, -2.25042720e-04,  1.25418720e-04]],

       [[ 2.06667238e-04,  2.35308004e-04,  6.89478100e-05, ...,
          3.94965040e-04,  1.92042166e-04,  1.03682356e-04],
        [ 2.59683124e-04,  3.66933652e-04,  2.68214416e-04, ...,
          3.18183412e-04,  5.23543798e